In [23]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate();
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [24]:
import tweepy as tw
import os
import json
import sys
import geocoder

In [13]:
# API Keys and Tokens
consumer_key = 'fRpq6Rk6LeDjJcq77AzR9vpFt'
consumer_secret = 'GeTDjPUQytrlKBx07KDqAKLTyRa7dfwP5JRqqKphmFujgx9Zx3'
access_token = '1423506172003291139-yQlvw5xX24vOmHSLCj1uKrYDuhQTKD'
access_secret = 'X2aRu8OMzNROksy4uIqLDf5hoMvXuR4xaEkYRYlyOxbdQ'


In [14]:
# Authorization and Authentication
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth)

In [15]:
# WOEID of delhi, india
woeid = 20070458

In [16]:
# fetching the trends
Trends = api.trends_place(id = woeid)

In [17]:
for value in Trends:
     print(value)
        #for trend in value['trends']:
      #  print(trend['name'], trend['tweet_volume'])

{'trends': [{'name': '#DominosPizza', 'url': 'http://twitter.com/search?q=%23DominosPizza', 'promoted_content': None, 'query': '%23DominosPizza', 'tweet_volume': None}, {'name': 'Rohit', 'url': 'http://twitter.com/search?q=Rohit', 'promoted_content': None, 'query': 'Rohit', 'tweet_volume': 22205}, {'name': '#FreedomFromDrugs', 'url': 'http://twitter.com/search?q=%23FreedomFromDrugs', 'promoted_content': None, 'query': '%23FreedomFromDrugs', 'tweet_volume': 342637}, {'name': '#5YearsWithAGUSTD', 'url': 'http://twitter.com/search?q=%235YearsWithAGUSTD', 'promoted_content': None, 'query': '%235YearsWithAGUSTD', 'tweet_volume': 147339}, {'name': 'Sant Rampal Ji Maharaj', 'url': 'http://twitter.com/search?q=%22Sant+Rampal+Ji+Maharaj%22', 'promoted_content': None, 'query': '%22Sant+Rampal+Ji+Maharaj%22', 'tweet_volume': 323523}, {'name': 'Afghanistan', 'url': 'http://twitter.com/search?q=Afghanistan', 'promoted_content': None, 'query': 'Afghanistan', 'tweet_volume': 1550353}, {'name': 'Pujar

In [18]:
for value in Trends:
    rdd=spark.sparkContext.parallelize(value['trends'])
Trend_with_hashtag=rdd.toDF(sampleRatio=0.01)

In [19]:
Trend_with_hashtag.createOrReplaceTempView("Trend")
TD=spark.sql("select name, tweet_volume from Trend")
TD.show()

+--------------------+------------+
|                name|tweet_volume|
+--------------------+------------+
|       #DominosPizza|        null|
|               Rohit|       22205|
|   #FreedomFromDrugs|      342637|
|   #5YearsWithAGUSTD|      147339|
|Sant Rampal Ji Ma...|      323523|
|         Afghanistan|     1550353|
|              Pujara|       14543|
|#IndianIdolGreate...|        null|
|#விஜய்_ಮಾಸ್ಟರ್_TR...|      172381|
|9YEARS OF ETT HYS...|       14767|
|D-1 TO FIGHTorESCAPE|      129790|
|          शहनाज़ गिल|       39833|
|          Tajikistan|       15416|
|               Kohli|       35476|
|             America|      420967|
|               Biden|      872031|
|BIRDIES LOVE YOU ...|        null|
|              Sharia|       18091|
|       JEON JUNGKOOK|       33336|
|     राष्ट्रपति अशरफ|        null|
+--------------------+------------+
only showing top 20 rows



In [20]:
trends = api.trends_place(id = woeid,exclude = "hashtags")
for value in trends:
    print(value)

{'trends': [{'name': 'Rohit', 'url': 'http://twitter.com/search?q=Rohit', 'promoted_content': None, 'query': 'Rohit', 'tweet_volume': 22205}, {'name': 'Sant Rampal Ji Maharaj', 'url': 'http://twitter.com/search?q=%22Sant+Rampal+Ji+Maharaj%22', 'promoted_content': None, 'query': '%22Sant+Rampal+Ji+Maharaj%22', 'tweet_volume': 324699}, {'name': 'Afghanistan', 'url': 'http://twitter.com/search?q=Afghanistan', 'promoted_content': None, 'query': 'Afghanistan', 'tweet_volume': 1550353}, {'name': 'Pujara', 'url': 'http://twitter.com/search?q=Pujara', 'promoted_content': None, 'query': 'Pujara', 'tweet_volume': 14596}, {'name': '9YEARS OF ETT HYSTERIA', 'url': 'http://twitter.com/search?q=%229YEARS+OF+ETT+HYSTERIA%22', 'promoted_content': None, 'query': '%229YEARS+OF+ETT+HYSTERIA%22', 'tweet_volume': 14767}, {'name': 'D-1 TO FIGHTorESCAPE', 'url': 'http://twitter.com/search?q=%22D-1+TO+FIGHTorESCAPE%22', 'promoted_content': None, 'query': '%22D-1+TO+FIGHTorESCAPE%22', 'tweet_volume': 129790}, 

In [25]:
for value in trends:
    RDD = spark.sparkContext.parallelize(value['trends'])
trend_without_hashtag=RDD.toDF(sampleRatio=0.01)

In [26]:
trend_without_hashtag.createOrReplaceTempView("trend")
td=spark.sql("select name, tweet_volume from trend")
td.show()

+--------------------+------------+
|                name|tweet_volume|
+--------------------+------------+
|               Rohit|       22205|
|Sant Rampal Ji Ma...|      324699|
|         Afghanistan|     1550353|
|              Pujara|       14596|
|9YEARS OF ETT HYS...|       14767|
|D-1 TO FIGHTorESCAPE|      129790|
|          शहनाज़ गिल|       39922|
|          Tajikistan|       15565|
|               Kohli|       35502|
|             America|      420475|
|               Biden|      872031|
|BIRDIES LOVE YOU ...|        null|
|              Sharia|       18091|
|       JEON JUNGKOOK|       33336|
|     राष्ट्रपति अशरफ|        null|
|     drug free india|        null|
|             Islamic|       60845|
|         Gerd Muller|       40834|
|       Sant Garibdas|        null|
|   ARMY ALERT OnTFMA|       19491|
+--------------------+------------+
only showing top 20 rows



In [10]:
#sc.stop()